<h2> EHRA Dashboard </h2>

First we have to generate simulated data to fill the scattermap. We'll store it in the dataframe <code>airQual</code>

In [ ]:
## Use code from Annie

# Sanity check for valid data structure 
# airQual.head()

Before getting the plot to show, need to sign up for mapbox account to get mapbox token: https://account.mapbox.com/

In [1]:
import plotly.express as px
px.set_mapbox_access_token('pk.eyJ1IjoiZmN4LWtpbGlnIiwiYSI6ImNsMHIwaG5rODJmY2QzYnM1NW1rcHRjOHMifQ.1kdhIW8dwvtyNDFHOCyx5g')
'''
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,  width=700, height=800)
fig.show()
df.head()
'''

'\ndf = px.data.carshare()\nfig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",\n                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,  width=700, height=800)\nfig.show()\ndf.head()\n'

If we wanted to just export this interactive plotly figure at this point as an html file, we can simply use the following snippet of code:

In [2]:
## fig.write_html("testing.html")
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import os
import base64

<h3> Using Dash </h3>

Here we're relying on JupyterDash to view the dashboard within our Jupyter notebook. Here I'm only implementing the layout without any of the callback functionality so that we have framework to build the visualization.

We'll still need to figure out a way to make the map full size and figure out what kind of filtering functionality we might want. Later we'll need focus on customizing the actual visuals.

Deleting app configuration is a weird fix for JupyterDash update that makes some parts of it no longer completely compatible with Dash. We Run with  <code>  mode = "inline" </code> so we can view the dashboard within the Jupyter notebook itself.

In [ ]:
#del app.config._read_only["requests_pathname_prefix"]
#app.run_server(mode="inline")

# Run following code in the terminal to generate requirements.txt
#jupyter nbconvert --output-dir="./reqs" --to script EHRA_Dashboard.ipynb
#cd reqs
#pipreqs

In [3]:
import requests
from contextlib import closing
import csv

sim_url="https://raw.githubusercontent.com/KiLiG-fcx/design-clinic/main/revised_sim.csv"

'''
A simulation for hotspot
If there are already hotspot information in the given dataset, this part can be neglected
Create some hotspots based on the simulation data, and collect the information in the 'dropdown' list
'''

dropdown=[[],[],[],[]]
with closing(requests.get(sim_url, stream=True)) as r:
    f = (line.decode('utf-8') for line in r.iter_lines())
    reader = csv.reader(f, delimiter=',', quotechar='"')
    next(reader,None)
    for row in reader:
        for i in range(2,12):
            row[i]=int(row[i])
        row[-1]=float(row[-1])
        row[-2]=float(row[-2])
        monitorid=int(row[1])
        if (monitorid<13250):
            dropdown[0].append(row)
        elif (13250<=monitorid<13500):
            dropdown[1].append(row)
        elif (13500<=monitorid<13750):
            dropdown[2].append(row)
        else:
            dropdown[3].append(row)

In [4]:
def readcols(url):
    df=pd.read_csv(url)
    fields=[] # fields used for all dropdown choices
    for col in df.columns:
        fields.append(col)
    return (fields)

In [9]:
import statistics
def hotspotcsv(hotspot):
    '''
        hotspot: int
        number chosen by dropdown (in simulated data: 1-4)
        create new csv to generate a new range of latitude, longitude
    '''
    fields=readcols(sim_url)
    rows_data=dropdown[hotspot-1] # the list of data that will be used to create the csv
    df=pd.DataFrame(rows_data,columns=fields)
    #print(type(df['latitude'][1]))
    return df

In [17]:
def hotspot_plot(hotspot):
    df = hotspotcsv(hotspot)
    hotspot_fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="NO2 (ppb)",
                        center={'lat':statistics.median(df['latitude']),'lon':statistics.median(df['longitude'])},
                                    # center is used to change the map center position
                      color_continuous_scale=px.colors.cyclical.IceFire, zoom=10, width=900, height=800)
    #hotspot_fig.show()
    return hotspot_fig

In [19]:
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

logo = os.getcwd() + '/assets/EHRA_logo.jpeg'
encoded_image = base64.b64encode(open(logo, 'rb').read())

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "1rem 1rem",
    "background-color": "#f8f9fa",
}

FIGURE_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "margin-left": "16rem",
    "margin-right": "1rem"
}

NEWS_STYLE = {
    "position": "fixed",
    "top": "0rem",
    "padding-top": "3rem",
    "padding-left": "1rem",
    "left": 0,
    "bottom": 0,
    "margin-left": "61rem",
    "margin-right": "1rem",
    "width": "40rem",
    "background-color": "#f8f9fa",
}

sidebar = html.Div(
    [
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '220px'}),
        html.H4('Pollutant'),
        dcc.RadioItems(id = 'radio-item-1',
                        options = [dict(label = 'NO2', value = 'A'),
                                    dict(label = 'VOC', value = 'B'),
                                    dict(label = 'pm1', value = 'C'),
                                    dict(label = 'pm10', value = 'D'),
                                    dict(label = 'pm25', value = 'E')],
                        value = 'A',
                        labelStyle={'display': 'block'}),

        html.P(['Abbreviations', html.Br(),\
            'PM: Particulate Matter', html.Br(),\
            'NO2: Nitrogen Dioxide', html.Br(),\
                'VOC: Volatine Organic Compound', html.Br(),\
                    'PM10: PM<10um diameter', html.Br(),\
                        'PM25: PM<2.5um diameter']),
       
        html.H4(['Select hotspot:']),

        dcc.Dropdown(
            ['Elementary School', 'Factory', 'Middle School', 'Park'],
            'Select hotspots',
            id='hotspot-dropdown'
        ),
        html.P('These hotspots were selected based on the air quality pollutant data released to the government.'),
        dbc.Button("Upload my Flow Monitor Data", color="primary", className="me-1", 
        href='https://docs.google.com/forms/d/e/1FAIpQLSeO__LGt9egJ8AEhVXB5YlO5YOEwjl9NYudUqe7H_MyvJ-MGQ/viewform?usp=pp_url', target='_blank'),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(dcc.Graph(id = 'main-graph'), style = FIGURE_STYLE)
news = html.Div(html.H4('Related News & Information'), style = NEWS_STYLE)
app.layout = html.Div([sidebar, content, news])

@app.callback(
    Output('main-graph', 'figure'),
    Input('hotspot-dropdown','value'))

def update_figure(hotspotid):
    if (hotspotid=='Elementary School'):
        return hotspot_plot(1)
    elif (hotspotid=="Factory"):
        return hotspot_plot(2)
    elif (hotspotid=="Middle School"):
        return hotspot_plot(3)
    elif (hotspotid=="Park"):
        return hotspot_plot(4)
    return hotspot_plot(1)


## Deploy app -------------------------------------------------------------
del app.config._read_only["requests_pathname_prefix"]
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
